## LangChain Agent 

- An Agent in LangChain is an LLM-powered decision-maker that Chooses which tool to use, when to use it, and how to use it — all dynamically at runtime.

- LangChain Agents are built on the ReAct Technique ~Reasoning (COT) + Acting (Tools/Actions) = ReAct~

In [ ]:
# Loading Important Packages

import langchainhub
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv(), override=True)
print(".env file loaded")

from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import (
    DuckDuckGoSearchRun,
    DuckDuckGoSearchResults,
    WikipediaQueryRun,
)
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain import hub
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import ChatOpenAI

print("\nRequired packages loaded")

.env file loaded

Required packages loaded


In [ ]:
# Set the API Key
api_key = os.environ.get("OPENAI_API_KEY")

# Set the model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=api_key)  # type: ignore

# Prompt Template
template = """ 
Answer the following questions as best you can.
Questions: {q}
"""

prompt_template = PromptTemplate.from_template(template)

prompt = hub.pull("hwchase17/react")  # Load ReAct Prompt from LangChain hub
# print(type(prompt))
# print(prompt.input_variables)
print(prompt.template)


# Defining Various Tools {tools} so that LLM can use it.

## 1. Python REPL Tool (for executing Python Code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name="python_repl",
    func=python_repl.run,
    description="Useful when you need to use Python to answer a question. You should input Python Code.",
)

## 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=500)  # type: ignore
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name="wikipedia",
    func=wikipedia.run,
    description="Useful for when you need to look up a topic, country, or person on Wikipedia",
)

## 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name="duckduckgo",
    func=search.run,
    description="Useful for when you need to perform an internet search to find information that another tool can't provide.",
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Loading the ReAct Agent and initializing the agent
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,  # handles potential parsing errors for debugging purposes. Making agents more robust and less failures
    max_iterations=10,  # Stops getting stuck in loops.
)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Testing the React Agent

In [27]:
question = 'Generate the first 20 numbers in the Fibonacci Series.'

output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I should use Python to generate the Fibonacci Series.
Action: python_repl
Action Input: 
```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        fib_series.append(fib_series[i-1] + fib_series[i-2])
    return fib_series

fibonacci(20)
```I now know the final answer
Final Answer: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]

> Finished chain.


In [ ]:
print(output['input']) # Contains User Prompt

 
Answer the following questions as best you can.
Questions: Generate the first 20 numbers in the Fibonacci Series.



In [ ]:
print(output['output']) # Contains Agent Response

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]


In [ ]:
question_2 = "Who is the current President of India?"

output = agent_executor.invoke({"input": prompt_template.format(q=question_2)})



> Entering new AgentExecutor chain...
I should use the wikipedia tool to look up the current President of India.
Action: wikipedia
Action Input: "President of India"Page: President of India
Summary: The president of India  (ISO: Bhārata kē Rāṣṭrapati) is the head of state of the Republic of India. The president is the nominal head of the executive, the first citizen of the country, and the supreme commander of the Indian Armed Forces. Droupadi Murmu is the 15th and current president, having taken office on 25 July 2022.
The office of president was created when India's  constitution came into force and it became a republic on 26 January 1950. The president iI now know the final answer
Final Answer: The current President of India is Droupadi Murmu.

> Finished chain.


In [32]:
print(f"Agent Answers:\n{output['output']}")

Agent Answers:
The current President of India is Droupadi Murmu.
